In [ ]:
# Import Notebook Packages
import warnings
from urllib import request
from urllib.request import urlopen
from urllib.error import HTTPError

import re
from io import StringIO
import os
import re
import sys
from collections import OrderedDict

os.environ["USE_PYGEOS"] = "0"

import geopandas as gpd
import xarray as xr
import pandas as pd
import pathlib as pl
import numpy as np
import pyogrio

import netCDF4

import ipyleaflet

import branca
import branca.colormap as cm

import folium
from folium import Circle, Marker
from folium import plugins
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from ipywidgets import widgets

from ipyleaflet import Map, GeoJSON

# PyPRMS needs
from pyPRMS import Dimensions
from pyPRMS.metadata.metadata import MetaData
from pyPRMS import ControlFile
from pyPRMS import Parameters
from pyPRMS import ParameterFile
from pyPRMS.prms_helpers import get_file_iter, cond_check
from pyPRMS.constants import (
    DIMENSIONS_HDR,
    PARAMETERS_HDR,
    VAR_DELIM,
    PTYPE_TO_PRMS_TYPE,
    PTYPE_TO_DTYPE,
)
from pyPRMS.Exceptions_custom import ParameterExistsError, ParameterNotValidError
import networkx as nx
from collections.abc import KeysView

import pywatershed as pws

from rich.console import Console
from rich.progress import track
from rich.progress import Progress
from rich import pretty

pretty.install()
con = Console()

warnings.filterwarnings("ignore")

#### Adds:
import matplotlib as mplib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import datetime as dt

# from datetime import datetime

import ipyleaflet
from ipyleaflet import Map, GeoJSON

from folium import Choropleth
from folium.plugins import BeautifyIcon

import branca
import branca.colormap as cm

import plotly.graph_objects as go
import plotly
import plotly.subplots
import plotly.express as px

import dataretrieval.nwis as nwis


from NHM_helpers.efc import *
from NHM_helpers import NHM_helpers as helpers
from NHM_helpers.NHM_Assist_utilities import *
from NHM_helpers.NHM_hydrofabric import *


import jupyter_black

jupyter_black.load()

## Introduction
This notebook is designed for the user to setup NHM-Assist and a provided NHM subbasin model in a project workspace.  If you have not been provided a NHM subbasin model, default paths are set to an example model that can be downloaded from the USGS OSN storage pod following these steps.
1. Open up a miniforge prompt.
2. cd to the **NHM-Assist** folder
3. type `python pull_domain.py --name=willamette_river`

## National Hydrologic Model subabsin models
A sub-basin models for any domain within the NHM are extracted using an automated workflow that generates a complete set of PRMS input files that contain the data and parameters required for a NHM-PRMS model [Regan and others, 2018](https://pubs.usgs.gov/publication/tm6B9). This tool is written in the [Python language](https://www.python.org) and is designed to be run from the command line on [USGS high-performance computing resources](https://www.usgs.gov/advanced-research-computing). At this time, users do not need to download this software and instead can request a model subset following these steps:

1. Go to the web page [https://www.sciencebase.gov/catalog/item/5e29b87fe4b0a79317cf7df5](https://www.sciencebase.gov/catalog/item/5e29b87fe4b0a79317cf7df5)
2. Click the child item titled, [“GIS Features of the Geospatial Fabric for the National Hydrologic Model, version 1.1.”](https://www.sciencebase.gov/catalog/item/5e29d1a0e4b0a79317cf7f63)
3. Download attached files "GFv1.1.gdb.zip" and compare NHM segments to your area-of-interest.
4. Send an emial to pnorton@usgs.gov that includes the following:
   * Name, Email address, Organization, and optionally, Phone;
   * Using GFv1.1.gdb, include one or more national model segments (nhm_seg) associated with watershed outlet(s) points in your area-of-interest.
   * Include a short descriptive summary of your modeling application.

## **User Provided Information**
<font size=4>The user must provide and/or review information where <font color='green'>&#x270D;**Enter Information:**</font> appears in this notebook. 

#### &#x270D;<font color='green'>**Enter Information:**</font> **selected NHM subbasin model folder name**.
> The default is set to the example NHM subbasin model name, "20220819_oregon".

In [ ]:
subbasin = "willamette_river"

model_dir = pl.Path(f"domain_data/{subbasin}")

#### &#x270D;<font color='green'>**Enter Information:**</font> **GIS file format**. 
> The default format is a geopackage (**.gpkg**) but other formats such as ESRI shape file (**.shp**) may have been provided.

In [ ]:
GIS_format = ".gpkg"

#### &#x270D;<font color='green'>**Enter Information:**</font> **parameter file name**. 
> The default file name is **myparam.param**. Parameter files from other model calibrations, i.e. byHRU, byHW, byHWobs can be selected here.

In [ ]:
param_file = "myparam.param"

param_filename = f"{model_dir}/{param_file}"

#### &#x270D;<font color='green'>**Enter Information:**</font> **control file name**. 
> The default file name is **control.default.bandit**.

In [ ]:
control_file_name = "control.default.bandit"

control = pws.Control.load_prms(
    pl.Path(model_dir / control_file_name), warn_unused_options=False
)  # loads the control file for pywatershed functions

#### &#x270D;<font color='green'>**Enter Information:**</font> **Minimum streamflow observations (days)**. 
> Notebook 2 displays NWIS gages not listed the parameter file. These will be limted to number of streamflow observations (days) indicated below.

In [ ]:
nwis_gage_nobs_min = 90  # days

#### &#x270D;<font color='green'>**Enter Information:**</font> **List of parameters**. 
> Notebook 3 visualizes parameter values from the parameter file. Type the parameters you wish to visualize in the list(s) below. To view complete lists of parameters, copy/paste the functions below into a code block.
```python
bynhru_parameter_list(param_filename)
bynmonth_bynhru_parameter_list(param_filename)
bynsegment_parameter_list(param_filename)
```

In [ ]:
# Create list of calibration parameters
nhru_params = [
    "carea_max",
    "emis_noppt",
    "fastcoef_lin",
    "freeh2o_cap",
    "gwflow_coef",
    "potet_sublim",
    "rad_trncf",
    "slowcoef_sq",
    "smidx_coef",
    "smidx_exp",
    "snowinfil_max",
    "soil2gw_max",
    "soil_moist_max",
    "soil_rechr_max_frac",
    "ssr2gw_exp",
    "ssr2gw_rate",
]  # List any output param here.

nhru_nmonths_params = [
    "adjmix_rain",
    "cecn_coef",
    "jh_coef",
    "tmax_allrain_offset",
    "tmax_allsnow",
    "radmax",
    "rain_cbh_adj",
    "snow_cbh_adj",
    "tmax_cbh_adj",
    "tmin_cbh_adj",
]  # List any output param here.

#### &#x270D;<font color='green'>**Enter Information:**</font> **List of output variables**. 
> Notebooks 5 and 6 visualize model output variables from the output file file. List the output variables desired to visualize. To find a list of additional variables for each process, use ".get_variables()". Examples are below.

```python
pws.PRMSCanopy.get_variables()
pws.PRMSSnow.get_variables()
pws.PRMSRunoff.get_variables()
pws.PRMSSoilzone.get_variables()
pws.PRMSGroundwater.get_variables()
pws.PRMSChannel.get_variables()
pws.PRMSStarfit.get_variables()
pws.meta.find_variables([pws.PRMSChannel.get_variables()[2]])
```

In [ ]:
selected_output_variables = [
    "hru_actet",
    "seg_outflow",
    "recharge",
    "net_rain",
    "net_snow",
    "net_ppt",
    "sroff",
    "ssres_flow",
    "gwres_flow",
    "gwres_sink",
    "snowmelt",
    "gwres_stor",
    "gwres_stor_change",
    "ssres_stor",
    "unused_potet",
    "sroff_vol",
    "ssres_flow_vol",
    "gwres_flow_vol",
]

#### &#x270D;<font color='green'>**Enter Information:**</font> **Display output in calendar years (January 1st - December 31st) or water years (October 1st - September 30th)**. 
> Notebooks 5 and 6 visualize model output variables based upon calendar years or water years.
> <br>Defualt is water years set to **True**. (change to **False** is calenadr years are preeferred.

In [ ]:
water_years = True

#### All needed information has been provided above. Now the rest of the notebook will create the needed objects, paths and directories for NHM-Assist notebooks.
> You're <font size=5 color="red">**NOT FINISHED**</font> yet!</font> <font size=5 color="red">**SAVE YOUR NOTEBOOK**</font> to retain entered information!

## Workspace Setup
The NHM-Assist repository is designed to be placed in a project folder. Two subfolders in the the NHM-Assist repository have critical supporting documents.

1. The **data_dependencies** folder with needed supporting files not included in the NHM v1.1 data release [Markstrom and others, 2024](https://www.sciencebase.gov/catalog/item/626c0d67d34e76103cd2ce4a)

2. The **data_domain** folder contains the provided NHM subbasin model.
    The provided NHM subbasin model folder should contain:
    - **control.default.bandit**
    - **myparam.param**
    - **sf_data.nc**
    - **cbh.nc**
    - **GIS** folder containing
        - **model_nhru.shp**
        - **model_nsegment.shp**
        - **model_npoigages.shp**
        - and/or **model_layers.gpkg**

NHM-Assist will create additional files and folders in NHM subbasin model folder. These include:

- **default_gages.csv**
- **NWISgages.csv**
 - **tmin.nc**
- **tmax.nc**
- **prcp.nc**
- **model_output** folder
- **notebook_output_files** folder containing:
    - **Folium_maps** folder
    - **GeoDataFrames** folder
    - **nc_files** folder
    - **shapefiles** folder
**Note:** If subfolders do no exist, they will be created when needed.


In [ ]:
# Establish paths and file names

gages_file = model_dir / "gages.csv"
nwis_gages_file = model_dir / "NWISgages.csv"

output_netcdf_filename = model_dir / "notebook_output_files" / "nc_files" / "sf_efc.nc"

notebook_dir = pl.Path("./").resolve()
NHM_dir = f"{notebook_dir}/data_dependencies/NHM_v1_1"


prms_meta = MetaData().metadata  # loads metadata functions for pyPRMS
pdb = ParameterFile(
    param_filename, metadata=prms_meta, verbose=False
)  # loads parmaeterfile functions for pyPRMS

In [ ]:
# Create/verify Jupyter notebooks output folder and subfolders in the model directory.
out_dir = model_dir / "output"
if not pl.Path.exists(out_dir):
    pl.Path(out_dir).mkdir(parents=True, exist_ok=True)
else:
    pass

notebook_output_dir = pl.Path(model_dir / "notebook_output_files").resolve()
if not pl.Path.exists(notebook_output_dir):
    pl.Path(notebook_output_dir).mkdir(parents=True, exist_ok=True)
else:
    pass

Folium_maps_dir = pl.Path(notebook_output_dir / "Folium_maps").resolve()
if not pl.Path.exists(Folium_maps_dir):
    pl.Path(Folium_maps_dir).mkdir(parents=True, exist_ok=True)
else:
    pass

html_maps_dir = pl.Path(notebook_output_dir / "html_maps").resolve()
if not pl.Path.exists(html_maps_dir):
    pl.Path(html_maps_dir).mkdir(parents=True, exist_ok=True)
else:
    pass

html_plots_dir = pl.Path(notebook_output_dir / "html_plots").resolve()
if not pl.Path.exists(html_plots_dir):
    pl.Path(html_plots_dir).mkdir(parents=True, exist_ok=True)
else:
    pass

nc_files_dir = pl.Path(notebook_output_dir / "nc_files").resolve()
if not (notebook_output_dir / "nc_files").exists():
    (notebook_output_dir / "nc_files").mkdir(parents=True)
else:
    pass

In [ ]:
# Print messages to display
workspace_txt = f"NHM model domain: [bold black]{subbasin}[/bold black], parameter file: [bold black]{param_file}[/bold black]\nSimulation and observation data range: {pd.to_datetime(str(control.start_time)).strftime('%m/%d/%Y')} - {pd.to_datetime(str(control.end_time)).strftime('%m/%d/%Y')} (from [bold]{control_file_name}[/bold])."